In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
# 载入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# 运行次数
max_steps = 100
# 图片数量
image_num = 3000
# 文件路径，
DIR = 'F:\\JupyterRoot\\'
# 定义会话
sess = tf.Session()

# 载入图片，把一定数量的测试图片载入内存
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name='embedding')

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        
# 命名空间
with tf.name_scope('Input'):
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
    
# 显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input', image_shaped_input,100) # 放进去10张图片

with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784,10]),name='W')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)
        
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    
sess.run(tf.global_variables_initializer())

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
        
# 产生的metadata文件
if tf.gfile.Exists(DIR + 'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR + 'projector/projector/metadata.tsv') #如果该文件存在，先删除
with open(DIR + 'projector/projector/metadata.tsv','w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:image_num],1))  # 拿到测试集所有标签，写入一部分到文件中去 
    for i in range(image_num):
        f.write(str(labels[i]) + '\n')

# 合并所有summary
merged = tf.summary.merge_all()

# 定义一个writer，写入图
projector_writer = tf.summary.FileWriter(DIR + 'projector/projector',sess.graph)
#用来保存网络模型
saver = tf.train.Saver()
# 定义配置文件
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR + 'projector/projector/metadata.tsv'
embed.sprite.image_path = DIR + 'projector/data/mnist_10k_sprite.png'
# 切分图片
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer, config)

for i in range(max_steps):
    batch_xs,batch_ys = mnist.train.next_batch(1000)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary,_  = sess.run([merged, train_step], feed_dict={x:batch_xs,y:batch_ys}, options=run_options, run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,'step%03d' % i)
    projector_writer.add_summary(summary, i)
    
    if i%5 == 0:
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("Iter " + str(i) + ", Testing Accuracy= " + str(acc))
        
saver.save(sess, DIR + 'projector/projector/a_model.ckpt', global_step=max_steps)
projector_writer.close()
sess.close()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iter 0, Testing Accuracy= 0.6051
Iter 5, Testing Accuracy= 0.5991
Iter 10, Testing Accuracy= 0.6065
Iter 15, Testing Accuracy= 0.4686
Iter 20, Testing Accuracy= 0.4625
Iter 25, Testing Accuracy= 0.5245
Iter 30, Testing Accuracy= 0.5556
Iter 35, Testing Accuracy= 0.5635
Iter 40, Testing Accuracy= 0.5615
Iter 45, Testing Accuracy= 0.5727
Iter 50, Testing Accuracy= 0.5868
Iter 55, Testing Accuracy= 0.6133
Iter 60, Testing Accuracy= 0.6332
Iter 65, Testing Accuracy= 0.6463
Iter 70, Testing Accuracy= 0.6619
Iter 75, Testing Accuracy= 0.6712
Iter 80, Testing Accuracy= 0.6854
Iter 85, Testing Accuracy= 0.7051
Iter 90, Testing Accuracy= 0.723
Iter 95, Testing Accuracy= 0.7372
